### 卷积神经网络，可以解决全连接层存在的对于图像数据信息提取的问题，卷积层可以保持数据的形状不变，以3维的数据形式接受输入数据，可以正确理解图像等具有形状的数据。

In [1]:
class Convolution:#卷积层的实现
    def _init_(self,W,b,stride=1,pad=0):
        self.W=W
        self.b=b
        self.stride=stride
        self.pad=pad
    def forward(self,x):
        FN,C,FH,FW=self.W.shape
        N,C,H,W=x.shape
        out_h=int(1+(H+2*self.pad-FH)/self.stride)
        out_w=int(1+(W+2*self.pad-FW)/self.stride)
        
        col=im2col(x,FH,FW,self.stride,self.pad)#im2col根据滤波器大小，步幅，填充将输入数据展开为2维数据。
        col_w=self.W.reshape(FN,-1)
        out=np.dot(col,col_w)+self.b
        
        out=out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)#out(N,C,H,W)
        
        
        

In [2]:
class Pooling:#池化层的实现：将输入数据形状规模减小。
    def _init_(self,pool_h,pool_w,stride=1,pad=0):
        self.pool_h=pool_h
        self.pool_w=pool_w
        self.stride=stride
        self.pad=pad
    def forward(self,x):
        N,C,H,W=x.shape
        out_h=int(1+(H-self.pool.h)/self.stride)
        out_w=int(1+(W-self.pool.w)/self.stride)
        
        col=im2col(x,self.pool.h,self.pool.w,self.stride,self.pad)
        col=col.shape(-1,self.pool.h*self.pool.w)
        
        out=np.max(col,axis=1)
        out=out.shape(N,out_h,out_w,C).transpose(0,3,1,2)

In [5]:
class SimpleConvNet:#CNN的实现
    def _init_(self,input_dim=(1,28,28),conv_param={'filter_num':30,'filter_size':5,'pad':0,'stride':1},hidden_size=100,output_size=10,weight_init_std=0.01):
        filter_num=conv_param['filter_num']
        filter_size=conv_param['filter_size']
        filter_pad=conv_param['pad']
        filter_stride=conv_param['stride']
        input_size=input_dim[1]
        
        conv_output_size=(input_size-filter_size+2*filter_pad)/filter_stride+1
        pool_output_size=int(filter_num*(conv_output_size/2)*(conv_output_size/2))
        
        self.params={}
        self.params['W1']=weight_init_std*np.random_randn(filter_num,input_dim[0],filter_size,filter_size)
        self.params['b1']=np.zeros(filter_num)
        self.params['W2']=weight_init_std*np.random_randn(pool_output_size,hidden_size)
        self.params['b2']=np.zeros(hidden_size)
        self.params['W3']=weight_init_std*np.random_randn(hidden_size,out_size)
        self.params['b3']=np.zeros(out_size)
        
        self.layers=Orderdict()
        self.layers['Conv1']=Convolution(self.params['W1'],self.params['b1'],conv_param['stride'],conv_params['pad'])
        self.layers['Relu1']=Relu()
        self.layers['Pool1']=Pooling(pool_h=2,pool_w=2,stride=2)
        self.layers['Affine1']=Affine(self.params['W2'],self.params['b2'])
        self.layers['Relu2']=Relu()
        self.layers['Affine2']=Affine(self.params['W3'],self.params['b3'])
        
        self.last_player=softmaxwithloss()
        
    def predict(self,x):
        for layer in self.layers.values():
            x=layer.forward(x)#按各层的顺序调用各层的forwar的()的方法。
            return x
        
    def loss(self,x,t):
        y=self.predict(x)
        return self.last_layer.forward(y,t)#softmaxwithloss()里的forward方法。
    
    def gradient(self,x,t):
        #forward
        sel.loss(x,t)
        
        #backward
        dout=1
        dout=self.last_layer.backward(dout)
        
        layers=list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout=layer.backward(dout)
            
        grads={}
        grads['W1']=self.layers['Conv1'].dw
        grads['b1']=self.layers['Conv1'].db
        grads['W2']=self.layers['Affine1'].dw
        grads['b2']=self.layers['Affine1'].db
        grads['W2']=self.layers['Affine2'].dw
        grads['b2']=self.layers['Affine2'].db
        
        return grads
    #更新
    network=SimpleConvNet()
    for key in('W1','b1','W2','b2','W3','b3'):
        network.params[key]-=learning_rate*grads[key]
        